<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: enforce_primitive_limits_and_compute_u0.c

## Authors: Leo Werneck, Zach Etienne, and Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain the function `enforce_primitive_limits_and_compute_u0`. This function uses the GRHayL simulation and equation-of-state parameters along with the metric to enforce limits on the computed primitives. It also output $u^0$, which is an input for the function `compute_conservs_and_Tmunu`.

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#enforce_limits_on_primitives_and_compute_u0): **The `enforce_limits_on_primitives_and_compute_u0` function**
    1. [Step 2.a](#enforce_limits_on_primitives): *Enforcing physical limits on the primitive variables*
        1. [Step 2.a.i](#hybrid_eos_limits): *Limits for the Hybrid EOS*
        1. [Step 2.a.ii](#tabulated_eos_limits): *Limits for the Tabulated EOS*
    1. [Step 2.b](#speed_limit_v): *Speed limit $v^i$ and compute $u^0$*

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss how we adjust the primitive variables to enforce simulation limits after the primitives have been computed by a conservative-to-primitive routine.

This function is usually used immediately after the new primitives have been computed, either by atmospheric reset or a conservative-to-primitive routine (e.g. TODO: link to multimethod). The outputs for this are also chosen so that the `compute_conservs_and_Tmunu` function can run afterwards with minimal additional computation.

<a id='enforce_limits_on_primitives_and_compute_u0'></a>

# Step 2: `enforce_limits_on_primitives_and_compute_u0` function \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives_and_compute_u0}$$

First, let's look at the arguments of `enforce_primitive_limits_and_compute_u0`:
```
void enforce_primitive_limits_and_compute_u0(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      primitive_quantities *restrict prims,
      int *restrict speed_limit)
```
The first two arguments appear in most GRHayL functions and contain information on the simulation and equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges. The final input is the primitive_quantities that the limits will be applied to.

The outputs are a primitive_quantities struct containing the primitives that have had floor and ceiling limits applied. The zeroth component of the 4-velocity, $u^0$, is also computed and stored in `prims`. The integer `speed_limit` provides diagnostic information on whether or not speed limits were applied to the velocity.

<a id='enforce_limits_on_primitives'></a>

## Step 2.a: Enforcing physical limits on the primitive variables \[Back to [top](#toc)\]
$$\label{enforce_limits_on_primitives}$$

We first apply the simplest limits by enforcing that
$$\rho_\mathrm{min} <= \rho_b <= \rho_\mathrm{max}$$

<a id='hybrid_eos_limits'></a>

### Step 2.a.i: Limits for the Hybrid EOS \[Back to [top](#toc)\]
$$\label{hybrid_eos_limits}$$

We then apply the limits on pressure. Since hybrid and tabulated EOS require different treatment, the code splits between the two. For hybrid EOS, if
$$P < 0.9*P_\mathrm{cold}$$
then the pressure is floored to
$$P = 0.9*P_\mathrm{min}$$
The factor of 0.9 helps with stability, as setting the condition to be exactly at P_cold can cause undesirable increases in the star pressure that behave better by allowing the pressure to fall slightly below P_cold.

On the other end, the upper limit depends on the metric. If $\psi^6$ is greater than the GRHayL parameter $\psi^6_\mathrm{threshold}$, then
$$P_\mathrm{max} = 1e5*P_\mathrm{cold}$$
Otherwise,
$$P_\mathrm{max} = 100*P_\mathrm{cold}$$
The resulting limit is if
$$P>P_\mathrm{max}~\mathrm{and~either}~\rho<100*\rho_{atm}~\mathrm{or}~\psi^6 > \psi^6_\mathrm{threshold}$$
then
$$P = P_\mathrm{max}$$

<a id='tabulated_eos_limits'></a>

### Step 2.a.ii: Limits for the Tabulated EOS \[Back to [top](#toc)\]
$$\label{tabulated_eos_limits}$$

TODO: Tabulated isn't in yet!

<a id='speed_limit_v'></a>

## Step 2.b: Speed limit $v^i$ and compute $u^0$ \[Back to [top](#toc)\]
$$\label{speed_limit_v}$$

Finally, we use the `limit_v_and_compute_u0` function to speed limit $v^i$ and compute $u^0$. For more on this function, please see 
TODO: link to nb